In [1]:
import numpy as np
import pandas as pd
import os
import datetime

In [2]:
f = open('../path_to_data.txt', "r")
path_to_dataset = os.path.join(f.read())
f.close() 

In [3]:
mydir = os.path.join(path_to_dataset, 'real_rain_gauge_data', 'csv_formatado/')
filename_list = os.listdir(mydir)
print(len(filename_list))

609


In [4]:

locs = pd.read_pickle(os.path.join('../reference/','rain_gauge_locs.pckl'))

In [5]:
def extractLonLat(filename):
    lonlat = filename.split("W")[1][:-4].split("_S")
    lon = -float(lonlat[0])
    lat = -float(lonlat[1])
    return(lon, lat)

In [6]:
st = '2020-12-01'
et = '2021-01-01'

In [47]:

i=0
for filename in filename_list:
    print(i)
    lon, lat = extractLonLat(filename)
    
    ind = str(locs.loc[np.stack([locs['lon'] == lon, locs['lat'] == lat]).all(axis=0)].index[0])
    # if(ind=='424'):
    #    print(ind)
    #    print(lon, lat)
    df = pd.read_csv(mydir+filename, delimiter = ',', header=0)
    df.DATA_UTC = pd.to_datetime(df.DATA_UTC, format='%Y-%m-%d')
    mask = (df.DATA_UTC >= st)&(df.DATA_UTC < et)
    nf = df.loc[mask]
    #print(ind)
    if(len(nf.DATA_UTC)>0):
        #print(nf.DATA_UTC.iloc[0])
        for date in nf.DATA_UTC:
            #print('date ', date)
            tmp = nf[nf.DATA_UTC==date].drop(columns='DATA_UTC').T
            tmp.columns = [ind]
            tmp_t = [date+pd.Timedelta(hours=int(i)) for i in list(tmp.index)]
            tmp['timestamp'] = tmp_t
            #tmp = tmp.set_index('timestamp')
            #print(tmp)
            if(date==nf.DATA_UTC.iloc[0]):
                h = tmp.copy()
            else:
                h = h.append(tmp, ignore_index=True)

        if(i==0):
            tot = h.copy()
        else:
            tot = pd.merge(tot, h, how='outer', on=['timestamp', 'timestamp']) #on=["timestamp"]) #pd.concat([tot, h], on='timestamp')

        #print(tot)

        i+=1
print(tot)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
166
167
168
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
270
271
272

In [49]:
col_names = list(tot.columns)
col_names.remove('timestamp')
list1 = [int(x) for x in col_names]

In [51]:

list1.sort()
list2 = [str(elem) for elem in list1]
tot = tot.reindex(['timestamp']+list2, axis=1)

In [52]:
tot

,timestamp,0,1,2,3,4,5,6,7,8,...,599,600,601,602,603,604,605,606,607,608
0,2020-12-01 00:00:00,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0
1,2020-12-01 01:00:00,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
2,2020-12-01 02:00:00,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0
3,2020-12-01 03:00:00,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0
4,2020-12-01 04:00:00,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2020-12-31 19:00:00,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
740,2020-12-31 20:00:00,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.2
741,2020-12-31 21:00:00,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.2
742,2020-12-31 22:00:00,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [53]:
#tot.to_pickle(os.path.join(path_to_dataset,'real_rain_gauge_data', 'rain_gauges.pckl'))

In [ ]:
#tot['438']

In [ ]:
#np.isnan(tot).sum(axis=0)

In [ ]:
#tot.columns[(np.isnan(tot)==True).sum(axis=0)/tot.shape[0] < 0.95]

In [ ]:
tot.shape

In [ ]:
#tot['530'].sum()

In [ ]:
#tot.reindex(list2, axis=1)['530'].sum()